In [30]:
import pandas as pd

json_loc = "/home/santiagovargas/dev/bondnet/bondnet/dataset/"
json_file = json_loc + "rev_corrected_bonds_qm_9_hydro_training.json"
pandas_file = pd.read_json(json_file)

In [36]:
#pandas_file["product_structure"][0]#["molecule"]
# for evans dataset it's just bond list

pandas_file.iloc[0].reactant_bonds
#pandas_file.columns

[[0, 1],
 [0, 9],
 [1, 10],
 [1, 2],
 [1, 11],
 [2, 13],
 [2, 3],
 [2, 12],
 [3, 14],
 [3, 5],
 [3, 4],
 [4, 5],
 [4, 15],
 [5, 16],
 [5, 6],
 [6, 7],
 [6, 8],
 [6, 17],
 [7, 19],
 [7, 18],
 [7, 8],
 [20, 21],
 [20, 22]]

In [16]:
import pandas as pd

json_loc = "/home/santiagovargas/dev/bondnet/bondnet/dataset/mg_dataset/"
json_file = json_loc + "20220613_reaction_data.json"
pandas_file = pd.read_json(json_file)

In [29]:
pandas_file.columns
#pandas_file.iloc[0]["product_structure"].keys()
#['graphs']#['adjacency']
#product_molecule_graph
#product_structure

Index(['_id', 'builder_meta', 'charge', 'spin_multiplicity', 'natoms',
       'elements', 'nelements', 'composition', 'formula_alphabetical',
       'chemsys', 'symmetry', 'reaction_id', 'reactant_id', 'product_id',
       'transition_state_id', 'deprecated', 'reactant_structure',
       'reactant_molecule_graph', 'reactant_molecule_graph_nometal',
       'reactant_bonds', 'reactant_bonds_nometal', 'reactant_energy',
       'reactant_zpe', 'reactant_enthalpy', 'reactant_entropy',
       'reactant_free_energy', 'product_structure', 'product_molecule_graph',
       'product_molecule_graph_nometal', 'product_bonds',
       'product_bonds_nometal', 'product_energy', 'product_zpe',
       'product_enthalpy', 'product_entropy', 'product_free_energy',
       'transition_state_structure', 'transition_state_energy',
       'transition_state_zpe', 'transition_state_enthalpy',
       'transition_state_entropy', 'transition_state_free_energy', 'dE', 'dH',
       'dS', 'dG', 'dE_barrier', 'dH_barri

In [101]:
import numpy as np 

def parse_cp(lines, verbose = True):
    lines_split = [line.split() for line in lines]
    cp_bond, cp_atom = False, False
    cp_name = "null"
    cp_dict = {}

    if(lines_split[0][4] == '(3,-3)'): 
        cp_atom = True
        if verbose: print("atom cp")
    elif(lines_split[0][4] == '(3,-1)'): 
        cp_bond = True
        if verbose: print("bond cp")
    
    else: 
        print(lines_split[0])
        print("ring critical bond not implemented")
    cp_atom_conditionals = {
        "cp_num": ["----------------"],
        "ele_info": ["Corresponding", "nucleus:"],
        "pos_ang": ["Position", "(Angstrom):"],
        "Lagrangian_K": ["Lagrangian", "kinetic", "energy"],
        "Hamiltonian_K": ["Hamiltonian", "kinetic", "energy"],
        "e_density":["Energy", "density", "E(r)"],
        "lap_e_density":["Laplacian", "electron", "density:"] ,
        "e_loc_func":["Electron", "localization", "function"] ,
        "ave_loc_ion_E":["Average", "local", "ionization", "energy"],
        "delta_g_promolecular":["Delta-g", "promolecular"],
        "delta_g_hirsh":["Delta-g", "Hirshfeld"],
        "esp_nuc":["ESP" , "nuclear", "charges:"],
        "esp_e":["ESP", "electrons:"],
        "esp_total":["Total", "ESP:"],
        "grad_norm":["Components", "gradient", "x/y/z"],
        "lap_norm":["Components", "Laplacian", "x/y/z"],
        "eig_hess":["Eigenvalues", "Hessian:"],
        "det_hessian":["Determinant", "Hessian:"],
        "ellip_e_dens":["Ellipticity", "electron", "density:"],
        "eta":["eta", "index:"]
    }

    cp_bond_conditionals = {
        "cp_num": ["----------------"],
        "pos_ang": ["Position", "(Angstrom):"],
        "Lagrangian_K": ["Lagrangian", "kinetic", "energy"],
        "Hamiltonian_K": ["Hamiltonian", "kinetic", "energy"],
        "e_density":["Energy", "density", "E(r)"],
        "lap_e_density":["Laplacian", "electron", "density:"] ,
        "e_loc_func":["Electron", "localization", "function"] ,
        "ave_loc_ion_E":["Average", "local", "ionization", "energy"],
        "delta_g_promolecular":["Delta-g", "promolecular"],
        "delta_g_hirsh":["Delta-g", "Hirshfeld"],
        "esp_nuc":["ESP" , "nuclear", "charges:"],
        "esp_e":["ESP", "electrons:"],
        "esp_total":["Total", "ESP:"],
        "grad_norm":["Components", "gradient", "x/y/z"],
        "lap_norm":["Components", "Laplacian", "x/y/z"],
        "eig_hess":["Eigenvalues", "Hessian:"],
        "det_hessian":["Determinant", "Hessian:"],
        "ellip_e_dens":["Ellipticity", "electron", "density:"],
        "eta":["eta", "index:"]
    }

    if cp_atom:
        for ind, i in enumerate(lines_split):
            for k, v in cp_atom_conditionals.items():
                if all(x in i for x in v):
                    if(k == "cp_num"):                        
                        cp_dict[k] = int(i[2][:-1])

                    elif(k == 'pos_ang'): 
                        cp_dict[k] = [float(x) for x in i[2:]]

                    elif(k == 'ele_info'):
                        cp_dict["element"] = i[2].split("(")[1]
                        cp_dict["number"] = i[2].split("(")[0]
                        cp_name = cp_dict["number"] + "_" + cp_dict["element"]

                    elif(k == 'esp_total'):
                        cp_dict[k] = float(i[2])

                    elif(k == 'eig_hess'):
                        cp_dict[k] = np.sum(np.array([float(x) for x in i[-3:]]))

                    elif(k == 'grad_norm' or k == 'lap_norm'):
                        cp_dict[k] = float(lines_split[ind+2][-1])

                    else: 
                        cp_dict[k] = float(i[-1])
                    
                    cp_atom_conditionals.pop(k)
                    break

    elif cp_bond: 
        for ind, i in enumerate(lines_split):
            for k, v in cp_bond_conditionals.items():
                if all(x in i for x in v):
                    if(k == "cp_num"):                        
                        cp_dict[k] = int(i[2][:-1])
                        cp_name = str(cp_dict[k]) + "_bond"

                    elif(k == 'pos_ang'): 
                        cp_dict[k] = [float(x) for x in i[2:]]

                    elif(k == 'esp_total'):
                        cp_dict[k] = float(i[2])

                    elif(k == 'eig_hess'):
                        cp_dict[k] = np.sum(np.array([float(x) for x in i[-3:]]))

                    elif(k == 'grad_norm' or k == 'lap_norm'):
                        cp_dict[k] = float(lines_split[ind+2][-1])

                    else: 
                        cp_dict[k] = float(i[-1])
                    
                    cp_atom_conditionals.pop(k)
                    break

    else: print("error")

    return cp_name, cp_dict

    

def get_qtaim_descs(file = "./CPprop_1157_1118_1158.txt"):
    """
    file: str
        path to file
    """
    # read file
    cp_dict, ret_dict = {}, {}

    with open(file) as f:
        lines = f.readlines()
        lines = [line[:-1] for line in lines]
    
    # section lines into segments on ----------------
    track = 0 
    for ind, line in enumerate(lines): 
        if "----------------" in line:
            lines_segment = []
        lines_segment.append(line)
        if(ind < len(lines)-1):
            if "----------------" in lines[ind+1]:
                cp_dict[track] = lines_segment
                track += 1
        else: 
            cp_dict[track] = lines_segment


    #print(cp_dict[0])
    #print(parse_cp(cp_dict[45]))
    
    for k, v in cp_dict.items():
        ind_atom, cp_dict = parse_cp(v, verbose = False)
        ret_dict[ind_atom] = cp_dict

    return ret_dict

get_qtaim_descs()

['----------------', 'CP', '32,', 'Type', '(3,+1)', '----------------']
ring critical bond not implemented
error
['----------------', 'CP', '38,', 'Type', '(3,+1)', '----------------']
ring critical bond not implemented
error


{'15_H': {'cp_num': 1,
  'element': 'H',
  'number': '15',
  'pos_ang': [-1.52681915653, -2.016065473374, 0.055985443927],
  'Lagrangian_K': 0.007046767867,
  'Hamiltonian_K': 3.115864634,
  'e_density': -3.115864634,
  'lap_e_density': -12.43527146,
  'e_loc_func': 0.9998529256,
  'ave_loc_ion_E': 0.5527507319,
  'delta_g_promolecular': 0.1114435137,
  'delta_g_hirsh': 0.2209817081,
  'esp_nuc': 32.13960419,
  'esp_e': -15.30006844,
  'esp_total': 16.83953575,
  'grad_norm': 8.345989734e-16,
  'lap_norm': -12.43527146,
  'eig_hess': -12.435271464,
  'det_hessian': -70.72843812,
  'ellip_e_dens': 0.001711,
  'eta': -1.157443},
 '3_C': {'cp_num': 2,
  'element': 'C',
  'number': '3',
  'pos_ang': [-1.640699559878, -1.038197636208, -0.378398475648],
  'Lagrangian_K': 5.628718839,
  'Hamiltonian_K': 64564.2311,
  'e_density': -64564.2311,
  'lap_e_density': -258234.4095,
  'e_loc_func': 0.9999994348,
  'ave_loc_ion_E': 9.902405806,
  'delta_g_promolecular': 0.0420953635,
  'delta_g_hirsh'